In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df_train_est_A = pd.read_csv('train_est_A.csv')
df_train_obs_A = pd.read_csv('train_obs_A.csv')
df_test_A = pd.read_csv('X_test_A.csv')
df_train_est_A = df_train_est_A.rename(columns={'Unnamed: 0': 'date_forecast'})

df_train_obs_A = df_train_obs_A.rename(columns={'Unnamed: 0': 'date_forecast'})

df_test_A = df_test_A.rename(columns={'Unnamed: 0': 'date_forecast'})

df_train_est_A.set_index('date_forecast', inplace=True)
df_train_obs_A.set_index('date_forecast', inplace=True)
df_test_A.set_index('date_forecast', inplace=True)
df_train_obs_A.index = pd.to_datetime(df_train_obs_A.index)
df_train_obs_A.index = pd.to_datetime(df_train_obs_A.index)

In [3]:
df_train_est_A=df_train_est_A.drop(columns=['date_calc'])

In [4]:
df = pd.concat([df_train_obs_A,df_train_est_A])

In [5]:
import re
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df_test_A = df_test_A.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [6]:
df.index = pd.to_datetime(df.index)
df['hours'] = df.index.hour
df['day'] = df.index.day
df['month'] = df.index.month
df['year'] = df.index.year
df_test_A.index = pd.to_datetime(df_test_A.index)
df_test_A['hours'] = df_test_A.index.hour
df_test_A['day'] = df_test_A.index.day
df_test_A['month'] = df_test_A.index.month
df_test_A['year'] = df_test_A.index.year
df_test_A = df_test_A.drop(columns = 'date_calc', axis = 1)
df['dayofyear'] = df.index.day_of_year
df_test_A['dayofyear'] = df_test_A.index.day_of_year

In [7]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
# Imputa i valori mancanti
df_knn =pd.DataFrame(imputer.fit_transform(df),columns = df.columns, index = df.index)

In [8]:
columns = df.drop(columns = 'pv_measurement').columns

In [9]:
from sklearn.preprocessing import MinMaxScaler
# Crea un oggetto MinMaaxScaler
scaler = MinMaxScaler()
# Normalizza il DataFrame
df_normalized = pd.DataFrame(scaler.fit_transform(df_knn[columns]),columns = df_knn[columns].columns, index = df_knn[columns].index)
df_normalized['pv_measurement'] = df[ 'pv_measurement']

In [10]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
# Sample DataFrame (replace this with your own dataset)
data = df_normalized
# Define the number of splits (e.g., 5 for 80-20 train-test splits)
n_splits = 3
# Initialize the TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=n_splits)

In [11]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_absolute_error

# Definisci il modello di machine learning
model = AdaBoostRegressor()

# Definisci il selezionatore di feature
rfe = RFE(estimator=model, n_features_to_select=None)

# Trova il numero migliore di feature in base al MAE
errors = []
for n_features in range(1, len(df.columns)):
    rfe.n_features_to_select = n_features
    print("Numero di Feature usate: ",n_features)
    # Calcola il MAE utilizzando la cross-validation
    maelist = []
    for train_index, test_index in tscv.split(data):
        train_data = data.iloc[train_index]
        test_data = data.iloc[test_index]

        # Extract target variable for training and testing data
        y_train = train_data['pv_measurement']
        y_test = test_data['pv_measurement']

        # Extract features for training and testing data
        X_train = train_data.drop(columns = 'pv_measurement')
        X_test = test_data.drop(columns = 'pv_measurement')
        
        # Train the XGBoost model
        rfe.fit(X_train, y_train)

        # Make predictions on the test data
        y_pred = rfe.predict(X_test)

        # Evaluate the model using Mean Absolute Error (MAE)
        mae = mean_absolute_error(y_test, y_pred)
        maelist.append(mae)
    # Calcola il MAE medio
    mean_mae = np.mean(maelist)
    print("MAE medio : ",mae)
    errors.append(mae)

Numero di Feature usate:  1
MAE medio :  355.1316092250611
Numero di Feature usate:  2
MAE medio :  283.86386923412846
Numero di Feature usate:  3
MAE medio :  303.00809024703227
Numero di Feature usate:  4
MAE medio :  278.9881896953671
Numero di Feature usate:  5
MAE medio :  278.7411140655558
Numero di Feature usate:  6
MAE medio :  279.4061718130589
Numero di Feature usate:  7
MAE medio :  300.8209772685974
Numero di Feature usate:  8
MAE medio :  292.00713881601405
Numero di Feature usate:  9
MAE medio :  293.6600676382466
Numero di Feature usate:  10
MAE medio :  284.2322074194553
Numero di Feature usate:  11
MAE medio :  299.3908840918212
Numero di Feature usate:  12
MAE medio :  302.8905895828467
Numero di Feature usate:  13
MAE medio :  303.44276430809487
Numero di Feature usate:  14
MAE medio :  296.891971737543
Numero di Feature usate:  15
MAE medio :  288.06933559530546
Numero di Feature usate:  16
MAE medio :  330.57395057021535
Numero di Feature usate:  17
MAE medio :  33

In [21]:
min_index = errors.index(min(errors))
errros_df = pd.DataFrame(errors)
n_features = min_index + 1
n_features

5

In [24]:
rfe.support_

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [14]:
data = df_normalized[selected_features]
data['pv_measurement'] = df_knn["pv_measurement"]

In [15]:
# Traina il modello sulle feature selezionate
model.fit(data[selected_features], data["pv_measurement"])
# Calcola il MAE con le feature selezionate
y_pred = model.predict(data[selected_features])
mae = mean_absolute_error(data["pv_measurement"], y_pred)
print("Il numero migliore di feature è:", n_features)
print("Il MAE con il numero di feature ottimale è:", mae)

Il numero migliore di feature è: 5
Il MAE con il numero di feature ottimale è: 332.4652202022095


In [16]:
from sklearn.decomposition import PCA
data = df_normalized
# Esegui una PCA
pca = PCA()
pca.fit(data.drop(columns  = 'pv_measurement'))

# Salva le componenti principali
components = pca.components_
param = {'verbose' : -1}

# Calcola l'errore di previsione per ogni numero di componenti
maelist = []
for n_components in range(1, len(components) + 1):
    for train_index, test_index in tscv.split(data):
        train_data = data.iloc[train_index]
        test_data = data.iloc[test_index]

        # Extract target variable for training and testing data
        y_train = train_data['pv_measurement']
        y_test = test_data['pv_measurement']

        # Extract features for training and testing data
        X_train = train_data.drop(columns = 'pv_measurement')
        X_test = test_data.drop(columns = 'pv_measurement')
        X_train_pca = pca.transform(X_train)[:, :n_components]
        X_test_pca = pca.transform(X_test)[:, :n_components]
        # Train the XGBoost model
        model = AdaBoostRegressor(**param)
        model.fit(X_train_pca, y_train)
        # Make predictions on the test data
        y_pred = model.predict(X_test_pca)

        # Evaluate the model using Mean Absolute Error (MAE)
        mae = mean_absolute_error(y_test, y_pred)
        print("Mean Absolute Error:", mae)
    maelist.append(mae)

# Trova il numero di componenti con il minimo errore
min_error_index = np.argmin(maelist)
n_components = min_error_index + 1

print("Il numero migliore di componenti è:", n_components)


TypeError: AdaBoostRegressor.__init__() got an unexpected keyword argument 'verbose'

In [ ]:
def objective(trial):
    n_estimators = trial.suggest_categorical("n_estimators", [50, 100, 200])
    learning_rate = trial.suggest_float("learning_rate", 0.1, 1.0)
    max_depth = trial.suggest_int("max_depth", 1, 10)
    model = AdaBoostRegressor(**param)
    maelist = []
    for train_index, test_index in tscv.split(data):
        train_data = data.iloc[train_index]
        test_data = data.iloc[test_index]

        # Extract target variable for training and testing data
        y_train = train_data['pv_measurement']
        y_test = test_data['pv_measurement']

        # Extract features for training and testing data
        X_train = train_data.drop(columns = 'pv_measurement')
        X_test = test_data.drop(columns = 'pv_measurement')
        
        # Train the XGBoost model
        model.fit(X_train, y_train,eval_set=[(X_test,y_test)])

        # Make predictions on the test data
        y_pred = model.predict(X_test)

        # Evaluate the model using Mean Absolute Error (MAE)
        mae = mean_absolute_error(y_test, y_pred)
        print("Mean Absolute Error:", mae)
        maelist.append(mae)
    # Return MAE
    mean_mae = np.mean(maelist)

    return mean_mae

In [ ]:
import optuna
import warnings
warnings.filterwarnings("ignore")
from warnings import simplefilter
simplefilter("ignore", category=RuntimeWarning)
study = optuna.create_study(direction='minimize')

In [ ]:
study.optimize(objective, n_trials=50)

In [ ]:
trial = study.best_trial

print("  Value: {}".format(trial.value))
print(trial.params)

In [ ]:
param = {'n_estimators': 648, 'reg_alpha': 0.18734665099633893, 'reg_lambda': 0.15271069876903318, 'colsample_bytree': 0.5, 'subsample': 0.8, 'learning_rate': 0.011131612016333637, 'max_depth': 27, 'num_leaves': 177, 'min_child_samples': 122, 'min_data_per_groups': 75}

In [ ]:
model_A = AdaBoostRegressor(**param)

In [ ]:
X_train = df.drop(columns = 'pv_measurement')
y_train = df['pv_measurement']

In [ ]:
model_A.fit(X_train,y_train)
df_test_A = df_test_A
y_pred_A = model_A.predict(df_test_A)

In [ ]:
plt.plot(range(len(y_pred_A)), y_pred_A, label='Predizione', color='red', linestyle='-')

# Aggiungi una legenda
plt.legend()

# Aggiungi etichette agli assi
plt.ylabel('Valori y')

# Titolo del grafico
plt.title('Grafico di Predizione')

# Mostra il grafico
plt.show()

In [ ]:
df = pd.DataFrame({'Prediction':y_pred_A})

In [ ]:
threshold = 0.05
df['Prediction'] = df['Prediction'].apply(lambda x: 0 if x < threshold else x)

In [ ]:
df.to_csv('A1.csv')

In [ ]:
df